In [ ]:
import json, numpy as np, tensorflow as tf  
from tensorflow import keras

#다운로드 시간이 오래 걸림. 이미지 6만장...
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.squeeze()
y_test = y_test.squeeze()

CAT, DOG = 3, 5
train_mask = np.isin(y_train, [CAT, DOG])
test_mask = np.isin(y_test, [CAT, DOG])

x_train_cd, y_train_cd = x_train[train_mask], y_train[train_mask]
x_test_cd, y_test_cd = x_test[test_mask], y_test[test_mask]

y_train_cd = (y_train_cd == DOG).astype("int32")   #0~255
y_test_cd = (y_test_cd == DOG).astype("int32")

x_train_cd = x_train_cd.astype("float32") / 255.0   #0.0~1.0 로 정규화
x_test_cd = x_test_cd.astype("float32") / 255.0

from sklearn.model_selection import train_test_split
x_train_cd, x_val_cd, y_train_cd, y_val_cd = train_test_split(
    x_train_cd, y_train_cd, test_size=0.2, random_state=42, stratify=y_train_cd
)

print("train: ", x_train_cd.shape, "val: ", x_val_cd.shape, "test: ", x_test_cd.shape)

classes = ["cat", "dog"]
with open("classes.json", "w", encoding="utf-8") as f:
    json.dump(classes, f, ensure_ascii=False)

print("Saved", "classes.json", classes)

c:\python3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\python3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\python3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 209s 1us/step
train:  (8000, 32, 32, 3) val:  (2000, 32, 32, 3) test:  (2000, 32, 32, 3)
Saved classes.json ['cat', 'dog']


In [ ]:
from tensorflow.keras import layers, models

data_augmentation = models.Sequential([   #인공신경망 모형
    layers.RandomFlip("horizontal")   #가로회전
], name="augument")

model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),   #입력 데이터 32x32 3 컬러
    data_augmentation, 

    layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
    #합성곱신경망 필터 3x3, 활성화함수   양수 그대로, 음수 0   출력사이즈 같게
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D(),   #최대값 선택

    layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    layers.GlobalAveragePooling2D(),   #평균값  선택

    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),   #20% 출력 off
    layers.Dense(1, activation="sigmoid")   #0.0~1.0  확률 0.5 크면 1 작으면 0
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
#             최적화(오차줄이는)  손실  오차계산            평가기준   정확도
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augument (Sequential)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,569 (396.75 KB)

 Trainable params: 101,569 (396.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#학습
EPOCHS = 10   #10번해라
BATCH_SIZE = 64   #

history = model.fit(
    x_train_cd, y_train_cd,
    validation_data=(x_val_cd, y_val_cd),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val_cd, y_val_cd, verbose=0)
test_loss, test_acc = model.evaluate(x_test_cd, y_test_cd, verbose=0)
print(f"val_acc={val_acc:.4f} test_acc={test_acc:.4f}")

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.5023 - loss: 0.6933 - val_accuracy: 0.5310 - val_loss: 0.6914
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.5434 - loss: 0.6870 - val_accuracy: 0.5620 - val_loss: 0.6829
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5828 - loss: 0.6722 - val_accuracy: 0.5775 - val_loss: 0.6743
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6026 - loss: 0.6575 - val_accuracy: 0.5545 - val_loss: 0.6877
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6202 - loss: 0.6467 - val_accuracy: 0.6145 - val_loss: 0.6453
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6596 - loss: 0.6155 - val_accuracy: 0.6655 - val_loss: 0.6163
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6756 - loss: 0.6023 - val_accuracy: 0.6660 - val_loss: 0.6107
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.6994 - loss: 0.5819 - val_accu

In [4]:
EPOCHS = 6
BATCH_SIZE = 64

history = model.fit(
    x_train_cd, y_train_cd,
    validation_data=(x_val_cd, y_val_cd),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

val_loss, val_acc = model.evaluate(x_val_cd, y_val_cd, verbose=0)
test_loss, test_acc = model.evaluate(x_test_cd, y_test_cd, verbose=0)
print(f"val_acc={val_acc:.4f} test_acc={test_acc:.4f}")

Epoch 1/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.7312 - loss: 0.5375 - val_accuracy: 0.7160 - val_loss: 0.5522
Epoch 2/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7360 - loss: 0.5301 - val_accuracy: 0.7030 - val_loss: 0.5435
Epoch 3/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7533 - loss: 0.5102 - val_accuracy: 0.7345 - val_loss: 0.5130
Epoch 4/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7542 - loss: 0.5056 - val_accuracy: 0.7200 - val_loss: 0.5232
Epoch 5/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.7555 - loss: 0.4995 - val_accuracy: 0.7535 - val_loss: 0.5023
Epoch 6/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7671 - loss: 0.4825 - val_accuracy: 0.7380 - val_loss: 0.5136
val_acc=0.7380 test_acc=0.7580


In [5]:
model.save("simple_cats_dogs.h5")
print(f"모델 저장 완료")

모델 저장 완료
